In [1]:
#import necessary libraries
import pandas as pd
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to C:\Users\Kirby
[nltk_data]     Wenceslao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Extraction of Filipino Tweets
#Refers to opening the raw data containing the Filipino Tweets
#into the python program

data_path = "data.csv"
#using pandas
df = pd.read_csv(data_path)
df

,Tweet Content,Sentiment,Label
0,Ayuda para sa lahat ni Aling Leni Robredo\nA)\...,Negative,Hate
1,FUTURE PUPPET NG MGA OLIGARCHS AT DRUGLORD! Le...,Negative,Hate
2,Sino ngayon ang tanso? #votebetterin2022\n htt...,Negative,Hate
3,A day after Leni Robredo painted the town pink...,Negative,Hate
4,Former senator Antonio Trillanes IV on Friday ...,Negative,Hate
...,...,...,...
19007,Pakiusap 'wag po natin sayangin ang nag-iisang...,Positive,Non-hate
19008,One Big Fight! Atin na 'to!\nhttps://t.co/jBy4...,Positive,Non-hate
19009,"tears of a true leader , she's Leni Rob...",Positive,Non-hate
19010,Handa na akong magiging susunod na Presidente ...,Positive,Non-hate


In [6]:
tweets = df['Tweet Content'].values
list_dup = set(tweets)
print("duplicates: ", 19012 - len(list_dup))
print(len(list_dup))

duplicates:  170
18842


In [3]:
df.Label.value_counts()

Non-hate    16446
Hate         2566
Name: Label, dtype: int64

In [4]:
df.Sentiment.value_counts()

Neutral     10284
Positive     6162
Negative     2566
Name: Sentiment, dtype: int64

In [5]:
groupedby_sentiment = df.groupby(df.Sentiment)
df_positive = groupedby_sentiment.get_group("Positive")
df_positive

,Tweet Content,Sentiment,Label
12850,LOOK: Beautiful Ladies inside and out Janella ...,Positive,Non-hate
12851,Eto ang totoo\nSa lahat ng kandidato\nSa pagka...,Positive,Non-hate
12852,how do people even have the audacity not to re...,Positive,Non-hate
12853,"@ABSCBNNews Go,,,go,,,pres,Leni robredo",Positive,Non-hate
12854,Malungkot sa pagpanaw ng CHR chair Chito Gasco...,Positive,Non-hate
...,...,...,...
19007,Pakiusap 'wag po natin sayangin ang nag-iisang...,Positive,Non-hate
19008,One Big Fight! Atin na 'to!\nhttps://t.co/jBy4...,Positive,Non-hate
19009,"tears of a true leader , she's Leni Rob...",Positive,Non-hate
19010,Handa na akong magiging susunod na Presidente ...,Positive,Non-hate


In [6]:
df_negative = groupedby_sentiment.get_group("Negative")
df_negative

,Tweet Content,Sentiment,Label
0,Ayuda para sa lahat ni Aling Leni Robredo\nA)\...,Negative,Hate
1,FUTURE PUPPET NG MGA OLIGARCHS AT DRUGLORD! Le...,Negative,Hate
2,Sino ngayon ang tanso? #votebetterin2022\n htt...,Negative,Hate
3,A day after Leni Robredo painted the town pink...,Negative,Hate
4,Former senator Antonio Trillanes IV on Friday ...,Negative,Hate
...,...,...,...
2561,putanginv sabi ng kapatid ko kung wala daw yun...,Negative,Hate
2562,@Mira_cleketuy the philippine national electio...,Negative,Hate
2812,"Hay grabe ka ISKO MORENO DOMAGOSO,\nWag na wag...",Negative,Hate
2906,Baka lang may mag-isip na porque I'm championi...,Negative,Hate


In [7]:
df_neutral = groupedby_sentiment.get_group("Neutral")
df_neutral

,Tweet Content,Sentiment,Label
2563,PLS PLS PLS DON'T CONNECT ALL COLOR PINK YOU S...,Neutral,Non-hate
2564,"@dearmistermuse Bongbong Marcos, Isko Moreno, ...",Neutral,Non-hate
2565,"Supporters ni VP Leni Robredo, nag motorcade s...",Neutral,Non-hate
2566,Kung hindi kasalanan ng anak ang mga nagawa ng...,Neutral,Non-hate
2567,Verrrry interesting that @IskoMoreno No. 3 in ...,Neutral,Non-hate
...,...,...,...
12845,Ang Presidente \n LENI ROBREDO\nBise Pr...,Neutral,Non-hate
12846,"May 07, 2022 Trending: In recent weeks, legion...",Neutral,Non-hate
12847,Ang Presidente-Leni Robredo\nBise Presidente-K...,Neutral,Non-hate
12848,Ang presidente LENI ROBREDO Bise presidente KI...,Neutral,Non-hate


In [8]:
#binary hate non-hate
df_hate = df_negative.sample(n = 2560)

df_positive = df_positive.sample(n = 1280)
df_neutral = df_neutral.sample(n = 1280)

df_nonhate = df_positive.append(df_neutral)

df_dataset = df_hate.append(df_nonhate)

In [9]:
df_dataset.to_csv('dataset.csv', index=False)
df_dataset

,Tweet Content,Sentiment,Label
2302,Worst Bong ever. https://t.co/QA7R8VYppC,Negative,Hate
1166,what i dont like about leni robredo's platform...,Negative,Hate
2294,Ito ang tunay na survey ni VP Leni Robredo #1 ...,Negative,Hate
1690,(3) BBM sued for Pork Barrel Scam\n\nhttps://t...,Negative,Hate
501,Sabog din sumagot tong si Norberto Gonzales no...,Negative,Hate
...,...,...,...
12314,President Leni Robredo and Vice President Kiko...,Neutral,Non-hate
11136,@jillrobredo 🌺🌺🌺\nthank you din kay @maraceped...,Neutral,Non-hate
7948,LOOK: Presidential candidate Bongbong Marcos m...,Neutral,Non-hate
3287,@itsmaxandcheese Leni Robredo for President 2022,Neutral,Non-hate


In [10]:
dataset = "dataset.csv"

In [11]:
#using csv reader only

file = open(dataset, encoding='utf-8')

csv_reader = csv.reader(file)

#skipping the header
header = next(csv_reader)

tweets = []
labels = []
sentiments = []
for row in csv_reader:
    #row_dict = {'Text': row[0].strip().replace("\n"," "), 'Label': row[2]}
    tweets.append(row[0].strip().replace("\n"," "))
    sentiments.append(row[1])
    labels.append(row[2])
    
for i in range(10):
    print('Text: ', tweets[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  Worst Bong ever. https://t.co/QA7R8VYppC 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  what i dont like about leni robredo's platforms is that she likes to play safe as to not upset most of her supporters which are middle and upper class citizens 💀 but then again that's just what neoliberalism really is about. to please the rich and pretend to care about the poor 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Ito ang tunay na survey ni VP Leni Robredo #1 sa pinaka-ayaw. Ganyan tal... https://t.co/rtAHaMtm1G via @YouTube 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  (3) BBM sued for Pork Barrel Scam  https://t.co/54QrShYrAR  (4) PCGG: Bongbong blocked return of $40M to gov’t  https://t.co/Q6LnavIFFD  BBM SINUNGALING:   Source: https://t.co/wGZwfG3rh8  (1) Oxford University Says Marcos Jr. Did Not Complete Degree  https://t.co/oujMmkanl9 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [12]:
#Converting the Dictionary from previous cell into a dataframe
xy = list(zip(tweets,labels))
Tweets_DF = pd.DataFrame(list(zip(tweets,labels)), columns=['Text', 'Label'])
Tweets_DF

,Text,Label
0,Worst Bong ever. https://t.co/QA7R8VYppC,Hate
1,what i dont like about leni robredo's platform...,Hate
2,Ito ang tunay na survey ni VP Leni Robredo #1 ...,Hate
3,(3) BBM sued for Pork Barrel Scam https://t.c...,Hate
4,Sabog din sumagot tong si Norberto Gonzales no...,Hate
...,...,...
5115,President Leni Robredo and Vice President Kiko...,Non-hate
5116,@jillrobredo 🌺🌺🌺 thank you din kay @maracepeda...,Non-hate
5117,LOOK: Presidential candidate Bongbong Marcos m...,Non-hate
5118,@itsmaxandcheese Leni Robredo for President 2022,Non-hate


In [13]:
#Data De-Identification
#Refers to the removal of the mentions in each tweet
#source regex https://stackoverflow.com/questions/2304632/regex-for-twitter-username


#data
tweets_de_identified = []

#insert process code
for tweet in tweets:
    output = ""
    sentence = tweet.split(" ")
    for part in sentence:
        if not re.match(r"(^|[^@\w])@(\w{1,15})\b", part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"

    tweets_de_identified.append(output)

for i in range(10):
    print('Text: ', tweets_de_identified[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  Worst Bong ever. https://t.co/QA7R8VYppC 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  what i dont like about leni robredo's platforms is that she likes to play safe as to not upset most of her supporters which are middle and upper class citizens 💀 but then again that's just what neoliberalism really is about. to please the rich and pretend to care about the poor 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Ito ang tunay na survey ni VP Leni Robredo #1 sa pinaka-ayaw. Ganyan tal... https://t.co/rtAHaMtm1G via 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  (3) BBM sued for Pork Barrel Scam  https://t.co/54QrShYrAR  (4) PCGG: Bongbong blocked return of $40M to gov’t  https://t.co/Q6LnavIFFD  BBM SINUNGALING:   Source: https://t.co/wGZwfG3rh8  (1) Oxford University Says Marcos Jr. Did Not Complete Degree  https://t.co/oujMmkanl9 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



In [14]:
#Uniform Resource Locator Removal
#The removal of urls/links such as media included in a tweet
#uses validators import 
#does not include strings that is not a hyper link which means that it will only remove urls that start with http or https,
#as those types of urls are clickable on tweets

#data
tweets_url_removed = []

#insert process code


for tweet in tweets_de_identified:
    output = ""
    sentence = tweet.split(" ")
    for part in sentence:
        valid = validators.url(part)
            
        if (not valid == True):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
    tweets_url_removed.append(output)

#output
for i in range(10):
    print('Text: ', tweets_url_removed[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  Worst Bong ever. 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  what i dont like about leni robredo's platforms is that she likes to play safe as to not upset most of her supporters which are middle and upper class citizens 💀 but then again that's just what neoliberalism really is about. to please the rich and pretend to care about the poor 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Ito ang tunay na survey ni VP Leni Robredo #1 sa pinaka-ayaw. Ganyan tal... via 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  (3) BBM sued for Pork Barrel Scam   (4) PCGG: Bongbong blocked return of $40M to gov’t   BBM SINUNGALING:   Source:  (1) Oxford University Says Marcos Jr. Did Not Complete Degree  

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Sabog din sumagot tong si Norberto Gonzales no? Padala daw tayo ng tropa to train in Russia before we sign anything with them. E hello, y

In [15]:
#Special Character Processing #Kirby
#Refers to the removal of special characters in a tweet that
#are standalone and not used as part of a word

#data
tweets_specialcharacters_removed = []

#insert process code

#remove emojis
#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
        
#remove special characters not part of a word 
#https://stackoverflow.com/questions/28860440/check-when-string-contains-only-special-characters-in-python

for tweet in tweets_url_removed:
    emoji_removed = emoji.replace_emoji(tweet, replace="[emoji]")
    sentence = emoji_removed.split(" ")
    output = ""       

    for part in sentence:
        if not (re.match(r"^[_\W]+$", part) or "[emoji]" in part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
    
    tweets_specialcharacters_removed.append(output)

#output
for i in range(10):
    print('Text: ', tweets_specialcharacters_removed[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  Worst Bong ever. 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  what i dont like about leni robredo's platforms is that she likes to play safe as to not upset most of her supporters which are middle and upper class citizens but then again that's just what neoliberalism really is about. to please the rich and pretend to care about the poor 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Ito ang tunay na survey ni VP Leni Robredo #1 sa pinaka-ayaw. Ganyan tal... via 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  (3) BBM sued for Pork Barrel Scam   (4) PCGG: Bongbong blocked return of $40M to gov’t   BBM SINUNGALING:   Source:  (1) Oxford University Says Marcos Jr. Did Not Complete Degree  

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  Sabog din sumagot tong si Norberto Gonzales no? Padala daw tayo ng tropa to train in Russia before we sign anything with them. E hello, you

In [16]:
filipino_stopwords = set(
    """
akin
aking
ako
alin
am
amin
aming
ang
ano
anumang
apat
at
atin
ating
ay
bababa
bago
bakit
bawat
bilang
dahil
dalawa
dapat
din
dito
doon
gagawin
gayunman
ginagawa
ginawa
ginawang
gumawa
gusto
habang
hanggang
hindi
huwag
iba
ibaba
ibabaw
ibig
ikaw
ilagay
ilalim
ilan
inyong
isa
isang
itaas
ito
iyo
iyon
iyong
ka
kahit
kailangan
kailanman
kami
kanila
kanilang
kanino
kanya
kanyang
kapag
kapwa
karamihan
katiyakan
katulad
kaya
kaysa
ko
kong
kulang
kumuha
kung
laban
lahat
lamang
likod
lima
maaari
maaaring
maging
mahusay
makita
marami
marapat
masyado
may
mayroon
mga
minsan
mismo
mula
muli
na
nabanggit
naging
nagkaroon
nais
nakita
namin
napaka
narito
nasaan
ng
ngayon
ni
nila
nilang
nito
niya
niyang
noon
o
pa
paano
pababa
paggawa
pagitan
pagkakaroon
pagkatapos
palabas
pamamagitan
panahon
pangalawa
para
paraan
pareho
pataas
pero
pumunta
pumupunta
sa
saan
sabi
sabihin
sarili
sila
sino
siya
tatlo
tayo
tulad
tungkol
una
walang
""".split()
)

In [17]:
#Normalization #Kirby
#Further cleaning of each tweet and removal of the topics
#which are the 10 Philippine National Candidates



#normalization process
#lowercase the tweet
#remove diacritics, numerics, english or filipino stopwords, leading and trailing symbols
#and candidate names

#data
tweets_normalized = []

#test
input = "LOWERCASE THIS dont , remove this name leni robredo, ñññ and bongbong, ?! 123 14"


#insert process code

#declare stopwords and candidate names
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
search = "leni robredo bongbong marcos isko moreno domagoso manny pacman pacquiao ping lacson ernie abella leody de guzman norberto gonzales jose montemayor jr faisal mangondato"
candidatelist = search.split(" ")

for tweet in tweets_url_removed:
    
    #lowercase the text
    lowercased_input = tweet.lower()

    #remove diacritics
    diacritics_removed = unidecode.unidecode(lowercased_input)

    sentence = diacritics_removed.split(" ")
    output = ""
    
    #remove numerical, stopwords, and candidate names
    for part in sentence:
        part = re.sub("[^A-Za-z ]+$", "", part)
        part = re.sub("^[^A-Za-z #]+", "", part)
        if not (len(part) <= 1 or re.match(r"[^a-zA-Z#]", part) or part in english_stopwords or part in filipino_stopwords or any(part in x for x in candidatelist)):     
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}" 
                
    tweets_normalized.append(output)

#output
for i in range(10):
    print('Text: ', tweets_normalized[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  worst ever 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  dont like robredo's platforms likes play safe upset supporters middle upper class citizens that's neoliberalism really please rich pretend care poor 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  tunay survey vp pinaka-ayaw ganyan tal via 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  bbm sued pork barrel scam pcgg blocked return gov't bbm sinungaling source oxford university says complete degree 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  sabog sumagot tong si padala daw tropa train russia sign anything hello need sign moa defense coop first training exchanges akala ba nsa dati #smnipresidentialdebate 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  si po literal tumakbo galing debate 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  hi bitch requested resea

In [18]:
#Hashtag Processing
#Removal of the hashtags contained in a tweet

#data
tweets_hashtags_removed = []


#insert process code

for tweet in tweets_normalized:
    output = ""
    sentence = tweet.split(" ")

    for part in sentence:
        if not re.match(r"#(\w+)", part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
    tweets_hashtags_removed.append(output)

#output
for i in range(10):
    print('Text: ', tweets_hashtags_removed[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  worst ever 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  dont like robredo's platforms likes play safe upset supporters middle upper class citizens that's neoliberalism really please rich pretend care poor 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  tunay survey vp pinaka-ayaw ganyan tal via 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  bbm sued pork barrel scam pcgg blocked return gov't bbm sinungaling source oxford university says complete degree 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  sabog sumagot tong si padala daw tropa train russia sign anything hello need sign moa defense coop first training exchanges akala ba nsa dati 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  si po literal tumakbo galing debate 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  hi bitch requested research upon searching can't

In [19]:
#Tokenization
#Segmentation of phrases into words

#data
tweets_tokenized = []

#insert process code
from nltk.tokenize import WhitespaceTokenizer

tokenizer = WhitespaceTokenizer()

for tweet in tweets_hashtags_removed:
    output = tokenizer.tokenize(tweet)
    tweets_tokenized.append(output)

#output
for i in range(125):
    print('Text: ', tweets_tokenized[i], "\n\nLabel: ", labels[i],"\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

Text:  ['worst', 'ever'] 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  ['dont', 'like', "robredo's", 'platforms', 'likes', 'play', 'safe', 'upset', 'supporters', 'middle', 'upper', 'class', 'citizens', "that's", 'neoliberalism', 'really', 'please', 'rich', 'pretend', 'care', 'poor'] 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  ['tunay', 'survey', 'vp', 'pinaka-ayaw', 'ganyan', 'tal', 'via'] 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  ['bbm', 'sued', 'pork', 'barrel', 'scam', 'pcgg', 'blocked', 'return', "gov't", 'bbm', 'sinungaling', 'source', 'oxford', 'university', 'says', 'complete', 'degree'] 

Label:  Hate 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Text:  ['sabog', 'sumagot', 'tong', 'si', 'padala', 'daw', 'tropa', 'train', 'russia', 'sign', 'anything', 'hello', 'need', 'sign', 'moa', 'defense', 'coop', 'first', 'training', 'exchanges', 'akala', 'ba', 'nsa', 'dati'] 

Label:  Hate 


In [20]:
print(len(tweets_tokenized), len(labels))

5120 5120


In [21]:
#x = tweets_tokenized, y = labels
x = tweets_tokenized
y = labels

In [22]:
#Data Splitting and Saving

#70-30 split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, test_size=0.3)

#Saving data to be used on other files
x_train_file = open('7030/x_train.pkl', 'wb')
pickle.dump(x_train,x_train_file)
x_train_file.close()

x_test_file = open('7030/x_test.pkl', 'wb')
pickle.dump(x_test,x_test_file)
x_test_file.close()

y_train_file = open('7030/y_train.pkl', 'wb')
pickle.dump(y_train,y_train_file)
y_train_file.close()

y_test_file = open('7030/y_test.pkl', 'wb')
pickle.dump(y_test,y_test_file)
y_test_file.close()

tweet_collection_file = open('7030/tweet_collection.pkl', 'wb')
pickle.dump(tweets_tokenized,tweet_collection_file)
tweet_collection_file.close()

In [23]:
#Data Splitting and Saving

#80-20 split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

#Saving data to be used on other files
x_train_file = open('8020/x_train.pkl', 'wb')
pickle.dump(x_train,x_train_file)
x_train_file.close()

x_test_file = open('8020/x_test.pkl', 'wb')
pickle.dump(x_test,x_test_file)
x_test_file.close()

y_train_file = open('8020/y_train.pkl', 'wb')
pickle.dump(y_train,y_train_file)
y_train_file.close()

y_test_file = open('8020/y_test.pkl', 'wb')
pickle.dump(y_test,y_test_file)
y_test_file.close()

tweet_collection_file = open('8020/tweet_collection.pkl', 'wb')
pickle.dump(tweets_tokenized,tweet_collection_file)
tweet_collection_file.close()

In [24]:
#Data Splitting and Saving

#90-10 split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.9, test_size=0.1)

#Saving data to be used on other files
x_train_file = open('9010/x_train.pkl', 'wb')
pickle.dump(x_train,x_train_file)
x_train_file.close()

x_test_file = open('9010/x_test.pkl', 'wb')
pickle.dump(x_test,x_test_file)
x_test_file.close()

y_train_file = open('9010/y_train.pkl', 'wb')
pickle.dump(y_train,y_train_file)
y_train_file.close()

y_test_file = open('9010/y_test.pkl', 'wb')
pickle.dump(y_test,y_test_file)
y_test_file.close()

tweet_collection_file = open('9010/tweet_collection.pkl', 'wb')
pickle.dump(tweets_tokenized,tweet_collection_file)
tweet_collection_file.close()